In [21]:
from pororo import Pororo
import pandas as pd
from sentence_transformers import util
import torch
import numpy as np

In [38]:
import enum
import os
import re
from sklearn.model_selection import train_test_split
import glob
import json
# from configs import DEFINES #config파일 사용위해
import csv
from tqdm import tqdm #진행속도를 알려줌

In [26]:
# def load_data():
#     question_path = './training/원천데이터/질문/알츠하이머병/치료'
#     answer_path = './training/원천데이터/답변/알츠하이머병/치료'
    
#     return glob(question_path + "/*.json"), glob(answer_path + "/*.json")

In [27]:
# q_data, a_data = load_data()

In [217]:
# def all_data():
#     question_path = './training/원천데이터/질문/알츠하이머병'
#     answer_path = './training/원천데이터/답변/알츠하이머병'
#     q_data=glob.glob(question_path + '/*/*.json')
#     a_data=glob.glob(answer_path + '/*/*.json')

#     return q_data, a_data

In [39]:
def all_data():
    question_path = './training/원천데이터/질문/알츠하이머병'
    answer_path = './training/원천데이터/답변/알츠하이머병'

    return glob.glob(question_path + '/*/*.json'), glob.glob(answer_path + '/*/*.json')

In [41]:
q_data,a_data = all_data()
q_data[6]

'./training/원천데이터/질문/알츠하이머병\\원인\\HC-Q-0332359.json'

In [42]:
len(q_data), len(a_data)

(1347, 3434)

In [43]:
with open(q_data[1000], 'r', encoding='utf-8') as file:
    json_data = json.load(file)
    print(json_data)

{'fileName': 'HC-Q-0332742', 'participantsInfo': {'participantID': 'QC015', 'gender': '남성', 'age': '50대 이상', 'occupation': '생산/노무직', 'history': False, 'rPlace': '부산/대구/울산/경상'}, 'disease_category': '뇌신경정신질환', 'disease_name': {'kor': '알츠하이머병', 'eng': "Alzheimer's disease"}, 'intention': '치료', 'question': '알츠하이머병 치료에는 어떤 방법들이 있는지 자세히 알고 싶어요.', 'entities': [{'id': 0, 'text': '알츠하이머병', 'entity': '질환명', 'position': 0}], 'num_of_words': 8}


In [44]:
q_list = []
for i in range(1000):
    with open(q_data[i], 'r', encoding='utf-8') as file:
        json_data = json.load(file)
        q_list.append(json_data['question'])
print(q_list)

['알츠하이머병의 원인으로 매일 소주를 섭취하는 것이 언급되고 있는데, 이에 대한 근거가 있는지 알려주세요.', '알츠하이머병이라는 질병은 유전적 영향을 받는 것인가요?', '알츠하이머병의 발생 원인에 대한 연구나 발견이 진행 중인가요?', '알츠하이머병의 발병과 관련하여 뇌의 노화로 인한 증상과 원인을 알려주세요.', '알츠하이머병의 원인과 관련된 연구 결과가 있을까요? 알려주세요.', '알츠하이머병의 원인은 무엇인가요?', '알츠하이머병이 발생하는 주요 원인은 무엇인가요?', '알츠하이머병 예방을 위해 매일 소주 한 병씩 섭취하는 것이 권장되는 이유가 무엇인가요?', '알츠하이머병과 다른 질병들이 발병하는 원인들에 대해 알려주세요.', '알츠하이머병이 발생하는데 영향을 미치는 주된 요인들은 어떤 것들이 있는지 알려주세요.', '알츠하이머병이 발생하는데에는 어떤 요인들이 영향을 미치나요?', '알츠하이머병과 치매 사이에는 어떠한 연관성이 있을까요?', '알츠하이머병의 원인 중 하나인 뇌의 노화에 대해서 자세히 알려주세요.', '알츠하이머병이 발생하는 주된 원인은 무엇인가요?', '알츠하이머병의 원인에는 가족력, 유전적 요인 이외에 다른 요인들이 있는지 자세히 알려주세요.', '알츠하이머병의 원인에 대해 상세하게 설명해주세요.', '알츠하이머병의 원인과 그에 따른 증상에 대해 알고 싶어요.', '알츠하이머병이 발생하는 주된 원인은 무엇인가요?', '알츠하이머병의 발생과정에서 어떤 요인이 가장 문제가 될 수 있는지 알려주세요.', '알츠하이머병과 관련된 감염균에 대한 정의와 특징을 알 수 있을까요?', '알츠하이머병의 원인이 30대에서 더 쉽게 찾아볼 수 있는 이유가 있나요?', '알츠하이머병의 원인에는 가족력, 유전적 요인 외에 다른 요인들이 있을까요?', '알츠하이머병의 원인과 관련된 다른 원인들을 알려주세요.', '알츠하이머병이 생기는 주된 원인은 무엇인가요?', '알츠하이머병과 다른 노화 현상들 사이에는 어떤 차이점이 있을까요?', '알츠하이머병의 발병 

In [45]:
json_data['intention']

'치료'

In [46]:
a_list = []
for i in range(1000):
    with open(a_data[i], 'r', encoding='utf-8') as file:
        json_data = json.load(file)
        sentence = ""
        for key in json_data['answer']:
            sentence += json_data['answer'][key]
        a_list.append(sentence)
print(len(a_list))

1000


In [47]:
i_list=[]
for i in range(1000):
    with open(q_data[i],'r',encoding='utf-8') as file:
        json_data=json.load(file)
        sentence = ""
        sentence += json_data['intention']
        i_list.append(sentence)


In [48]:
print(i_list)

['원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인', '원인

In [49]:
q_df = pd.DataFrame(q_list)
a_df = pd.DataFrame(a_list)
df = pd.concat((q_df, a_df), axis=1)
df.columns=['question', 'answer']

In [50]:
df[990:]

,question,answer
990,알츠하이머병을 완치하는 데에 어떤 종류의 치료가 필요한가요?,"알츠하이머병은 노인들 사이에서 흔하게 발생하는 뇌의 퇴행성 질환으로, 아밀로이드와 ..."
991,알츠하이머병을 치료하기 위한 다른 치료 방법들이 있는지 궁금합니다.,"알츠하이머병은 서서히 진행되는 질병으로, 인지기능 저하와 뇌 기능 손상을 특징으로 ..."
992,알츠하이머병을 치료하는 데에 있어서 어떤 종류의 의료 전문가를 찾아야 할까요?,알츠하이머병은 진행하는 치매의 주요 원인이다. 이 질병은 아밀로이드 베타 단백질이 ...
993,알츠하이머병을 치료하기 위해 현재 사용되는 치료법에 대해 설명해주세요.,"알츠하이머병은 치매를 일으키는 퇴행성 뇌 질환으로, 진행이 빠르게 진행되는 특징을 ..."
994,알츠하이머병의 증상을 줄이기 위한 치료 방법 중 어떤 것이 가장 효과적인지 알려주세요.,알츠하이머병은 노인들에게서 가장 흔히 발생하는 치매의 원인 중 하나입니다. 신경세포...
995,알츠하이머병을 초기에 치료하면 어떤 점이 좋을까요?,알츠하이머병은 주로 노인들에서 발견되는 치매의 가장 흔한 형태입니다. 이 질병은 최...
996,알츠하이머병을 치료하기 위한 다른 치료법은 없을까요?,"알츠하이머병은 뇌 조직의 이상으로 인한 퇴행성 신경 질환으로, 주로 노년층에서 발생..."
997,알츠하이머병 치료에 있어서 가장 효과적인 방법은 무엇인가요?,알츠하이머병은 노인들에게서 흔히 발생하는 치매의 한 형태입니다. 아밀로이드 단백질과...
998,알츠하이머병을 치료하는데 있어서 어떤 종류의 약물이 사용되는지 자세히 알려주세요.,알츠하이머병은 노인들이 흔히 겪는 치매를 일으키는 퇴행성 신경질환입니다. 뇌세포에 ...
999,알츠하이머병을 치료하기 위해 주로 사용되는 치료법은 무엇인가요?,알츠하이머병은 현재까지의 많은 연구를 통해 예방과 조기 진단을 통해 진행을 억제할 ...


In [48]:
# q_df = pd.DataFrame(q_list)
# a_df = pd.DataFrame(a_list)
# i_df = pd.DataFrame(i_list)
# df = pd.concat((q_df, a_df,i_df), axis=1)
# df.columns=['question', 'answer','intention']

In [51]:
df.insert(loc=2, column='intention', value=i_list)

In [52]:
df[:100]

,question,answer,intention
0,"알츠하이머병의 원인으로 매일 소주를 섭취하는 것이 언급되고 있는데, 이에 대한 근거...","알츠하이머병의 정확한 원인은 아직 밝혀지지 않았지만, 연구들이 알츠하이머병의 발병 ...",원인
1,알츠하이머병이라는 질병은 유전적 영향을 받는 것인가요?,알츠하이머병은 현재까지 완전한 원인이 밝혀지지 않았습니다.알츠하이머병은 아직 완전히...,원인
2,알츠하이머병의 발생 원인에 대한 연구나 발견이 진행 중인가요?,"알츠하이머병은 치매를 일으키는 가장 흔한 퇴행성 뇌질환으로, 1907년 독일 의사 ...",원인
3,알츠하이머병의 발병과 관련하여 뇌의 노화로 인한 증상과 원인을 알려주세요.,알츠하이머병은 현재까지 그 발병 원인에 대한 완벽한 해명은 아직 이루어지지 않았습니...,원인
4,알츠하이머병의 원인과 관련된 연구 결과가 있을까요? 알려주세요.,"알츠하이머병은 복잡한 질환으로, 아직도 원인이 완전히 밝혀진 것은 아닙니다. 그러나...",원인
...,...,...,...
95,"알츠하이머병과 기억 장애, 언어 장애의 원인이 서로 다른가요? 어떤 요소가 원인이 ...","알츠하이머병은 노인에게서 가장 흔하게 발생하는 치매 형태로서, 인지 기능의 저하와 ...",원인
96,알츠하이머병이 발생하는데 영향을 주는 주요한 원인은 무엇인가요?,"알츠하이머병은 신경계통의 치매로서, 다양한 원인들의 복합적인 작용에 의해 발생할 수...",원인
97,"알츠하이머병이 유전적인 원인에 의해 발생한다면, 그 원인이 어떤 역할을 하는지 알려...","알츠하이머병은 치매의 가장 흔한 형태 중 하나로, 원인을 정확히 알 수 없지만, 다...",원인
98,알츠하이머병과 치매 사이에는 어떠한 공통점과 차이점이 있는지 알고 싶어요.,알츠하이머병은 노인들에게서 흔히 발생하는 신경퇴행성 질환입니다. 이 질병은 기억력 ...,원인


In [53]:
df.iloc[450]

question                     알츠하이머병이 노망과 같은 증세를 가지고 있는 것이 맞나요?
answer       알츠하이머병은 노인성 치매의 주요 원인 중 하나로, 뇌 신경세포의 손상이 진행되면서...
intention                                                   정의
Name: 450, dtype: object

In [54]:
with open(a_data[1000], 'r', encoding='utf-8') as file:
    json_data = json.load(file)
    print(json_data)

{'fileName': 'HC-A-02138665', 'disease_category': '뇌신경정신질환', 'disease_name': {'kor': '알츠하이머병', 'eng': "Alzheimer's disease"}, 'department': ['정신건강의학과', '신경과', '신경외과'], 'intention': '정의', 'answer': {'intro': '알츠하이머병은 신경퇴행성질환의 일종으로, 뇌의 신경세포 소실로 인해 기억력이 손상되는 질환입니다. 이 질병은 서서히 진행되며, 환자는 자신의 기억력을 잃게 됩니다.', 'body': '알츠하이머병은 노인들 사이에서 가장 흔하게 발병하는 치매로, 기억상실과 같은 인지능력 저하가 특징입니다. 일부 환자는 새로운 기억이나 문제 해결 능력이 감소하는 특징을 보이며, 다른 환자들은 기억력을 잃은 후에도 언어기능이나 판단력에는 영향이 없을 수 있습니다.', 'conclusion': '알츠하이머병을 관리하기 위해 기억력과 인지능력을 보존하기 위해 다양한 치료 방법이 있습니다. 이러한 치료 전략은 환자의 인지기능 손실을 지연시키고 일상생활을 유지하는데 도움을 줄 수 있습니다.'}, 'num_of_words': 76}


In [55]:
with open(q_data[1000], 'r', encoding='utf-8') as file:
    json_data = json.load(file)
    print(json_data['question'])

알츠하이머병 치료에는 어떤 방법들이 있는지 자세히 알고 싶어요.


In [56]:
df.to_csv("Alz.csv", sep=',')
df.tail()

,question,answer,intention
995,알츠하이머병을 초기에 치료하면 어떤 점이 좋을까요?,알츠하이머병은 주로 노인들에서 발견되는 치매의 가장 흔한 형태입니다. 이 질병은 최...,치료
996,알츠하이머병을 치료하기 위한 다른 치료법은 없을까요?,"알츠하이머병은 뇌 조직의 이상으로 인한 퇴행성 신경 질환으로, 주로 노년층에서 발생...",치료
997,알츠하이머병 치료에 있어서 가장 효과적인 방법은 무엇인가요?,알츠하이머병은 노인들에게서 흔히 발생하는 치매의 한 형태입니다. 아밀로이드 단백질과...,치료
998,알츠하이머병을 치료하는데 있어서 어떤 종류의 약물이 사용되는지 자세히 알려주세요.,알츠하이머병은 노인들이 흔히 겪는 치매를 일으키는 퇴행성 신경질환입니다. 뇌세포에 ...,치료
999,알츠하이머병을 치료하기 위해 주로 사용되는 치료법은 무엇인가요?,알츠하이머병은 현재까지의 많은 연구를 통해 예방과 조기 진단을 통해 진행을 억제할 ...,치료


In [60]:
question = df['question']
answer = df['answer']
df['answer']

0      알츠하이머병의 정확한 원인은 아직 밝혀지지 않았지만, 연구들이 알츠하이머병의 발병 ...
1      알츠하이머병은 현재까지 완전한 원인이 밝혀지지 않았습니다.알츠하이머병은 아직 완전히...
2      알츠하이머병은 치매를 일으키는 가장 흔한 퇴행성 뇌질환으로, 1907년 독일 의사 ...
3      알츠하이머병은 현재까지 그 발병 원인에 대한 완벽한 해명은 아직 이루어지지 않았습니...
4      알츠하이머병은 복잡한 질환으로, 아직도 원인이 완전히 밝혀진 것은 아닙니다. 그러나...
                             ...                        
995    알츠하이머병은 주로 노인들에서 발견되는 치매의 가장 흔한 형태입니다. 이 질병은 최...
996    알츠하이머병은 뇌 조직의 이상으로 인한 퇴행성 신경 질환으로, 주로 노년층에서 발생...
997    알츠하이머병은 노인들에게서 흔히 발생하는 치매의 한 형태입니다. 아밀로이드 단백질과...
998    알츠하이머병은 노인들이 흔히 겪는 치매를 일으키는 퇴행성 신경질환입니다. 뇌세포에 ...
999    알츠하이머병은 현재까지의 많은 연구를 통해 예방과 조기 진단을 통해 진행을 억제할 ...
Name: answer, Length: 1000, dtype: object

In [57]:
import re

# 한글, 영어, 숫자, 공백, ?!.,을 제외한 나머지 문자 제거
korean_pattern = r'[^ ?,.!A-Za-z0-9가-힣+]'

# 패턴 컴파일
normalizer = re.compile(korean_pattern)
normalizer

re.compile(r'[^ ?,.!A-Za-z0-9가-힣+]', re.UNICODE)

In [61]:
print(f'수정 전: {question[20]}')
print(f'수정 후: {normalizer.sub("", question[20])}')
print(f'수정 전: {answer[20]}')
print(f'수정 후: {normalizer.sub("", answer[20])}')

수정 전: 알츠하이머병의 원인이 30대에서 더 쉽게 찾아볼 수 있는 이유가 있나요?
수정 후: 알츠하이머병의 원인이 30대에서 더 쉽게 찾아볼 수 있는 이유가 있나요?
수정 전: 알츠하이머병은 노화된 단백질이 뇌세포를 파괴하는 퇴행성 신경 질환입니다. 이 질환은 노화에 따른 뇌신경세포의 죽음으로 인해 발생하며, 정확한 원인은 알려져 있지 않습니다. 초기 증상은 경미한 기억 장애부터 언어 장애, 행동 변화 등으로 나타납니다.알츠하이머병은 점진적인 기억력 감퇴와 언어 장애, 실행증(실어증), 시공간능력 장애, 행동 및 정신행동 증상 등을 보입니다. 진행되면서 불안, 우울, 난폭성 등과 같은 정신증상도 나타날 수 있습니다.알츠하이머병은 치료가 불가능한 질환이지만, 조기에 진단을 받고 적절한 치료와 관리를 시작하는 것이 중요합니다. 이를 통해 삶의 질을 향상시키고 증상을 완화시킬 수 있습니다.
수정 후: 알츠하이머병은 노화된 단백질이 뇌세포를 파괴하는 퇴행성 신경 질환입니다. 이 질환은 노화에 따른 뇌신경세포의 죽음으로 인해 발생하며, 정확한 원인은 알려져 있지 않습니다. 초기 증상은 경미한 기억 장애부터 언어 장애, 행동 변화 등으로 나타납니다.알츠하이머병은 점진적인 기억력 감퇴와 언어 장애, 실행증실어증, 시공간능력 장애, 행동 및 정신행동 증상 등을 보입니다. 진행되면서 불안, 우울, 난폭성 등과 같은 정신증상도 나타날 수 있습니다.알츠하이머병은 치료가 불가능한 질환이지만, 조기에 진단을 받고 적절한 치료와 관리를 시작하는 것이 중요합니다. 이를 통해 삶의 질을 향상시키고 증상을 완화시킬 수 있습니다.


In [62]:
def normalize(sentence):
    return normalizer.sub("", sentence)

normalize(question[20])

'알츠하이머병의 원인이 30대에서 더 쉽게 찾아볼 수 있는 이유가 있나요?'

In [63]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [64]:
import konlpy.tag

In [65]:
from konlpy.tag import Okt
from mecab import MeCab

# 형태소 분석기
mecab = MeCab()
okt = Okt()

In [68]:
mecab.morphs(normalize(question[996]))

['알츠하이머병', '을', '치료', '하', '기', '위한', '다른', '치료법', '은', '없', '을까요', '?']

In [69]:
# 한글 전처리를 함수화
def clean_text(sentence, tagger):
    sentence = normalize(sentence)
    sentence = tagger.morphs(sentence)
    sentence = ' '.join(sentence)
    sentence = sentence.lower()
    return sentence

In [70]:
# 한글
clean_text(question[20], okt)
clean_text(answer[20], okt)

'알츠하이머병 은 노화 된 단백질 이 뇌 세포 를 파괴 하는 퇴 행성 신경 질환 입니다 . 이 질환 은 노화 에 따른 뇌신경 세포 의 죽음 으로 인해 발생 하며 , 정확한 원인 은 알려져 있지 않습니다 . 초기 증상 은 경미한 기억 장애 부터 언어 장애 , 행동 변화 등 으로 나타납니다 . 알츠하이머병 은 점진 적 인 기억 력 감퇴 와 언어 장애 , 실행증 실어증 , 시공간 능력 장애 , 행동 및 정신 행동 증상 등 을 보입니다 . 진행 되면서 불안 , 우울 , 난폭 성 등 과 같은 정신증 상도 나타날 수 있습니다 . 알츠하이머병 은 치료 가 불가능한 질환 이지만 , 조기 에 진단 을 받고 적절한 치료 와 관리 를 시작 하는 것 이 중요합니다 . 이를 통해 삶 의 질 을 향상 시키고 증상 을 완화 시킬 수 있습니다 .'

In [71]:
len(question), len(answer)

(1000, 1000)

In [72]:
questions = [clean_text(sent, okt) for sent in question.values[:1000]]
answers = [clean_text(sent, okt) for sent in answer.values[:1000]]

In [73]:
questions[:5]

['알츠하이머병 의 원인 으로 매일 소주 를 섭취 하는 것 이 언급 되고 있는데 , 이 에 대한 근거 가 있는지 알려주세요 .',
 '알츠하이머병 이라는 질병 은 유전 적 영향 을 받는 것 인가요 ?',
 '알츠하이머병 의 발생 원인 에 대한 연구 나 발견 이 진행 중 인가요 ?',
 '알츠하이머병 의 발병 과 관련 하여 뇌 의 노화 로 인한 증상 과 원인 을 알려주세요 .',
 '알츠하이머병 의 원인 과 관련 된 연구 결과 가 있을까요 ? 알려주세요 .']

In [74]:
answers[:5]

['알츠하이머병 의 정확한 원인 은 아직 밝혀지지 않았지만 , 연구 들 이 알츠하이머병 의 발병 기전 에 대해 논의 하고 있습니다 . 일부 연구 에 따르면 , 유전 적 인 요소 와 뇌 의 기능 손상 이 관련 되어 있다고 알려져 있습니다 . 알츠하이머병 은 아밀로이드 베타 단백질 과 타우 단백질 의 과도 한 생 성 , 뇌 세포 의 비정상 적 인 활동 , 뇌 조직 의 변화 로 인해 발생 하는 것 으로 생각 되고 있습니다 . 이러한 변화 가 알츠하이머병 의 발병 위험 을 증가 시키고 , 병 의 진행 을 가속 화 시킨다는 것 입니다 . 알츠하이머병 의 발병 과 관련 된 위험 요소 에 대해 서는 더 많은 연구 와 조사 가 필요합니다 . 더 많은 연구 와 자료 수집 을 통해 알츠하이머병 에 대한 더 많은 이해 와 예방 방법 이 개발 될 것 으로 기대 됩니다 .',
 '알츠하이머병 은 현재 까지 완전한 원인 이 밝혀지지 않았습니다 . 알츠하이머병 은 아직 완전히 이해 되지 않았지만 , 연구 결과 에 따르면 유전 적 인 요소 와 다양한 환경 적 인 요인 이 이 질환 을 일으키는 역할 을 한다고 알려져 있습니다 . 특히 , 아밀로이드 베타 단백질 의 비정상 적 인 축적 이 알츠하이머병 과 관련 이 있는 것 으로 알려져 있습니다 . 이 외 에도 나이 , 노화 , 고혈압 , 당뇨병 , 그리고 흡연 등 과 같은 다른 요인 들 도 알츠하이머병 발병 과 연관 성 이 있을 수 있습니다 . 더 많은 연구 와 조사 를 통해 알츠하이머병 의 원인 을 파악 하고 예방 방법 을 개발 할 필요 가 있습니다 .',
 '알츠하이머병 은 치매 를 일으키는 가장 흔한 퇴 행성 뇌 질환 으로 , 1907년 독일 의사 알 로이스 알츠하이머 에 의해 처음 으로 보고 되었습니다 . 이 질환 의 원인 에 대해 서는 현재 까지 명확한 답 은 없으나 , 치매 발생 의 위험 요소 와 관련 하여 몇 가지 위험 요인 이 알려져 있습니다 . 일반 적 으로 , 가장 잘 알려진 요인 중 하나 는 고령 입니다 . 고령 은 치매 

In [75]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [78]:
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2


class WordVocab():
    def __init__(self):
        self.word2index = {
            '<PAD>': PAD_TOKEN,
            '<SOS>': SOS_TOKEN, 
            '<EOS>': EOS_TOKEN,
        }
        self.word2count = {}
        self.index2word = {
            PAD_TOKEN: '<PAD>', 
            SOS_TOKEN: '<SOS>', 
            EOS_TOKEN: '<EOS>'
        }
        
        self.n_words = 3  # PAD, SOS, EOS 포함

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [80]:
questions[25]

'알츠하이머병 의 발병 원인 과 관련 된 주요 요인 들 은 무엇 인가요 ?'

In [81]:
print(f'원문: {questions[560]}')
lang = WordVocab()
lang.add_sentence(questions[560])
print('==='*10)
print('[단어사전]')
print('***'*10)
print(lang.word2index)

원문: 알츠하이머병 으로 인한 증상 중 에서 특히 주의 해야 할 증상 은 무엇 인가요 ?
[단어사전]
******************************
{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '알츠하이머병': 3, '으로': 4, '인한': 5, '증상': 6, '중': 7, '에서': 8, '특히': 9, '주의': 10, '해야': 11, '할': 12, '은': 13, '무엇': 14, '인가요': 15, '?': 16}


In [82]:
max_length = 10
sentence_length = 6

sentence_tokens = np.random.randint(low=3, high=100, size=(sentence_length,))
sentence_tokens = sentence_tokens.tolist()
print(f'Generated Sentence: {sentence_tokens}')

sentence_tokens = sentence_tokens[:(max_length-1)]

token_length = len(sentence_tokens)

# 문장의 맨 끝부분에 <EOS> 토큰 추가
sentence_tokens.append(2)

for i in range(token_length, max_length-1):
    # 나머지 빈 곳에 <PAD> 토큰 추가
    sentence_tokens.append(0)

print(f'Output: {sentence_tokens}')
print(f'Total Length: {len(sentence_tokens)}')

Generated Sentence: [92, 72, 49, 6, 87, 32]
Output: [92, 72, 49, 6, 87, 32, 2, 0, 0, 0]
Total Length: 10


In [83]:
from konlpy.tag import Mecab, Okt
import re

class TextDataset(Dataset):
    def __init__(self, csv_path, min_length=3, max_length=32):
        super(TextDataset, self).__init__()
        # data_dir = 'data'
        
        # TOKEN 정의
        self.PAD_TOKEN = 0 # Padding 토큰
        self.SOS_TOKEN = 1 # SOS 토큰
        self.EOS_TOKEN = 2 # EOS 토큰
        
        self.tagger = MeCab()   # 형태소 분석기
        self.max_length = max_length # 한 문장의 최대 길이 지정
        
        # CSV 데이터 로드
        # df = pd.read_csv(os.path.join(data_dir, csv_path))
        df=pd.read_csv('Alz.csv')
        # 한글 정규화
        korean_pattern = r'[^ ?,.!A-Za-z0-9가-힣+]'
        self.normalizer = re.compile(korean_pattern)
        
        # src: 질의, tgt: 답변
        src_clean = []
        tgt_clean = []
        
        # 단어 사전 생성
        wordvocab = WordVocab()
        
        for _, row in df.iterrows():
            src = row['question']
            tgt = row['answer']
            
            # 한글 전처리
            src = self.clean_text(src)
            tgt = self.clean_text(tgt)
            
            if len(src.split()) > min_length and len(tgt.split()) > min_length:
                # 최소 길이를 넘어가는 문장의 단어만 추가
                wordvocab.add_sentence(src)
                wordvocab.add_sentence(tgt)
                src_clean.append(src)
                tgt_clean.append(tgt)            
        
        self.srcs = src_clean
        self.tgts = tgt_clean
        self.wordvocab = wordvocab

    
    def normalize(self, sentence):
        # 정규표현식에 따른 한글 정규화
        return self.normalizer.sub("", sentence)

    def clean_text(self, sentence):
        # 한글 정규화
        sentence = self.normalize(sentence)
        # 형태소 처리
        sentence = self.tagger.morphs(sentence)
        sentence = ' '.join(sentence)
        sentence = sentence.lower()
        return sentence
    
    def texts_to_sequences(self, sentence):
        # 문장 -> 시퀀스로 변환
        return [self.wordvocab.word2index[w] for w in sentence.split()]

    def pad_sequence(self, sentence_tokens):
        # 문장의 맨 끝 토큰은 제거
        sentence_tokens = sentence_tokens[:(self.max_length-1)]
        token_length = len(sentence_tokens)

        # 문장의 맨 끝부분에 <EOS> 토큰 추가
        sentence_tokens.append(self.EOS_TOKEN)

        for i in range(token_length, (self.max_length-1)):
            # 나머지 빈 곳에 <PAD> 토큰 추가
            sentence_tokens.append(self.PAD_TOKEN)
        return sentence_tokens
    
    def __getitem__(self, idx):
        inputs = self.srcs[idx]
        inputs_sequences = self.texts_to_sequences(inputs)
        inputs_padded = self.pad_sequence(inputs_sequences)
        
        outputs = self.tgts[idx]
        outputs_sequences = self.texts_to_sequences(outputs)
        outputs_padded = self.pad_sequence(outputs_sequences)
        
        return torch.tensor(inputs_padded), torch.tensor(outputs_padded)
    
    def __len__(self):
        return len(self.srcs)

In [84]:
# 한 문장의 최대 단어길이를 25로 설정
MAX_LENGTH = 25

dataset = TextDataset('Alz.csv', min_length=3, max_length=MAX_LENGTH)

In [85]:
# 10번째 데이터 임의 추출
x, y = dataset[10]

In [86]:
print(f'x shape: {x.shape}')
print(x)

print(f'y shape: {y.shape}')
print(y)

x shape: torch.Size([25])
tensor([  3,  14,  77,  11,  12, 405,  21,  12, 410, 120,  40,  14, 109,  81,
        361, 422, 112,   2,   0,   0,   0,   0,   0,   0,   0])
y shape: torch.Size([25])
tensor([  3,  32, 113, 114, 115,  31,   5,  81, 346, 139, 161,  12, 108,  89,
         29, 201, 164, 165, 138,  90,  49,  50, 120,  40,   2])


In [87]:
# 80%의 데이터를 train에 할당합니다.
train_size = int(len(dataset) * 0.8)
train_size

800

In [88]:
# 나머지 20% 데이터를 test에 할당합니다.
test_size = len(dataset) - train_size
test_size

200

In [89]:
from torch.utils.data import random_split

# 랜덤 스플릿으로 분할을 완료합니다.
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

DataLoader생성
-배치 구성을 쉽게 하기 위해서 torch.utils.data.DataLoader를 활용합니다.
-train/test 데이터셋 모두 batch_size=16 으로 설정하겠습니다.

In [90]:
from torch.utils.data import DataLoader, SubsetRandomSampler

train_loader = DataLoader(train_dataset, 
                          batch_size=16, 
                          shuffle=True)

test_loader = DataLoader(test_dataset, 
                         batch_size=16, 
                         shuffle=True)

In [91]:
# 1개의 배치 데이터를 추출합니다.
x, y = next(iter(train_loader))

In [92]:
# shape: (batch_size, sequence_length)
x.shape, y.shape

(torch.Size([16, 25]), torch.Size([16, 25]))

모델/Encoder

In [93]:
class Encoder(nn.Module):
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers):
        super(Encoder, self).__init__()
        
        # 단어 사전의 개수 지정
        self.num_vocabs = num_vocabs
        # 임베딩 레이어 정의 (number of vocabs, embedding dimension)
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        # GRU (embedding dimension)
        self.gru = nn.GRU(embedding_dim, 
                          hidden_size, 
                          num_layers=num_layers, 
                          bidirectional=False)
        
    def forward(self, x):
        x = self.embedding(x).permute(1, 0, 2)
        output, hidden = self.gru(x)
        return output, hidden

In [94]:
# Embedding Layer의 입/출력 shape에 대한 이해

embedding_dim = 64 # 임베딩 차원
embedding = nn.Embedding(dataset.wordvocab.n_words, embedding_dim)

# x의 shape을 변경합니다.
# (batch_size, sequence_length) => (sequence_length, batch_size)
embedded = embedding(x)

print(x.shape)
print(embedded.shape)
# input:  (sequence_length, batch_size)
# output: (sequence_length, batch_size, embedding_dim)

torch.Size([16, 25])
torch.Size([16, 25, 64])


In [95]:
embedded = embedded.permute(1, 0, 2)
print(embedded.shape)
# (sequence_length, batch_size, embedding_dim)


torch.Size([25, 16, 64])


In [96]:
hidden_size = 32   

gru = nn.GRU(embedding_dim,      # embedding 차원
             hidden_size, 
             num_layers=1, 
             bidirectional=False)

# input       : (sequence_length, batch_size, embedding_dim)
# h0          : (Bidirectional(1) x number of layers(1), batch_size, hidden_size)
o, h = gru(embedded, None)

print(o.shape)
print(h.shape)
# output      : (sequence_length, batch_size, hidden_size x bidirectional(1))
# hidden_state: (bidirectional(1) x number of layers(1), batch_size, hidden_size)

torch.Size([25, 16, 32])
torch.Size([1, 16, 32])


In [97]:
NUM_VOCABS = dataset.wordvocab.n_words
print(f'number of vocabs: {NUM_VOCABS}')

number of vocabs: 2206


In [98]:
# Encoder 정의
encoder = Encoder(NUM_VOCABS, 
                  hidden_size=32, 
                  embedding_dim=64, 
                  num_layers=1)

In [99]:
# Encoder에 x 통과 후 output, hidden_size 의 shape 확인
# input(x)    : (batch_size, sequence_length)
o, h = encoder(x)

print(o.shape)
print(h.shape)
# output      : (sequence_length, batch_size, hidden_size x bidirectional(1))
# hidden_state: (bidirectional(1) x number of layers(1), batch_size, hidden_size

torch.Size([25, 16, 32])
torch.Size([1, 16, 32])


Decoder

In [100]:
class Decoder(nn.Module):
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers=1, dropout=0.2):
        super(Decoder, self).__init__()
        # 단어사전 개수
        self.num_vocabs = num_vocabs
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(embedding_dim, 
                          hidden_size, 
                          num_layers=num_layers, 
                          bidirectional=False)
        
        # 최종 출력은 단어사전의 개수
        self.fc = nn.Linear(hidden_size, num_vocabs)
        
    def forward(self, x, hidden_state):
        x = x.unsqueeze(0) # (1, batch_size) 로 변환
        embedded = F.relu(self.embedding(x))
        embedded = self.dropout(embedded)
        output, hidden = self.gru(embedded, hidden_state)
        output = self.fc(output.squeeze(0)) # (sequence_length, batch_size, hidden_size(32) x bidirectional(1))
        return output, hidden

In [101]:
#Embedding Layer의 입/출력 shape
x = torch.abs(torch.randn(size=(1, 16)).long())
print(x)
x.shape
# batch_size = 16 이라 가정했을 때,
# (1, batch_size)
# 여기서 batch_size => (1, batch_size) 로 shape 변환을 선행

tensor([[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0]])


torch.Size([1, 16])

In [102]:
embedding_dim = 64 # 임베딩 차원
embedding = nn.Embedding(dataset.wordvocab.n_words, embedding_dim)

embedded = embedding(x)
embedded.shape
# embedding 출력
# (1, batch_size, embedding_dim)

torch.Size([1, 16, 64])

In [103]:
 #GRU Layer의 입/출력 shape에 대한 이해
hidden_size = 32

gru = nn.GRU(embedding_dim, 
             hidden_size, 
             num_layers=1, 
             bidirectional=False, 
             batch_first=False, # batch_first=False로 지정
            )

o, h = gru(embedded)

print(o.shape)
# output shape: (sequence_length, batch_size, hidden_size(32) x bidirectional(1))
print(h.shape)
# hidden_state shape: (Bidirectional(1) x number of layers(1), batch_size, hidden_size(32))

torch.Size([1, 16, 32])
torch.Size([1, 16, 32])


In [104]:
# 최종 출력층(FC) shape에 대한 이해
fc = nn.Linear(32, NUM_VOCABS) # 출력은 단어사전의 개수로 가정

output = fc(o[0])

print(o[0].shape)
print(output.shape)
# input : (batch_size, output from GRU)
# output: (batch_size, output dimension)

torch.Size([16, 32])
torch.Size([16, 2206])


In [105]:
#인코더 -> 디코더 입출력 shape
decoder = Decoder(num_vocabs=dataset.wordvocab.n_words, 
                  hidden_size=32, 
                  embedding_dim=64, 
                  num_layers=1)

In [106]:
x, y = next(iter(train_loader))

o, h = encoder(x)

print(o.shape, h.shape)
# output      : (batch_size, sequence_length, hidden_size(32) x bidirectional(1))
# hidden_state: (Bidirectional(1) x number of layers(1), batch_size, hidden_size(32))

torch.Size([25, 16, 32]) torch.Size([1, 16, 32])


인코더(Encoder)로부터 생성된 hidden_state(h)와 SOS 토큰을 디코더(Decoder)의 입력으로 넣어줍니다

In [107]:
# ***************
x = torch.abs(torch.full(size=(16,), fill_value=SOS_TOKEN, dtype=torch.long))
print(x)
x.shape

# batch_size = 16 이라 가정(16개의 SOS 토큰)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


torch.Size([16])

In [108]:
embedding_dim = 64 # 임베딩 차원
embedding = nn.Embedding(dataset.wordvocab.n_words, embedding_dim)

embedded = embedding(x)
embedded.shape
# embedding 출력
# (1, batch_size, embedding_dim)

torch.Size([16, 64])

In [109]:
decoder_output, decoder_hidden = decoder(x, h)
decoder_output.shape, decoder_hidden.shape
# (batch_size, num_vocabs), (1, batch_size, hidden_size)

(torch.Size([16, 2206]), torch.Size([1, 16, 32]))

Seq2seq

In [140]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, inputs, outputs, teacher_forcing_ratio=0.5):
        # inputs : (batch_size, sequence_length)
        # outputs: (batch_size, sequence_length)
        
        batch_size, output_length = outputs.shape
        output_num_vocabs = self.decoder.num_vocabs
        
        # 리턴할 예측된 outputs를 저장할 임시 변수
        # (sequence_length, batch_size, num_vocabs)
        predicted_outputs = torch.zeros(output_length, batch_size, output_num_vocabs).to(self.device)
        
        # 인코더에 입력 데이터 주입, encoder_output은 버리고 hidden_state 만 살립니다. 
        # 여기서 hidden_state가 디코더에 주입할 context vector 입니다.
        # (Bidirectional(1) x number of layers(1), batch_size, hidden_size)
        _, decoder_hidden = self.encoder(inputs)
        
        # (batch_size) shape의 SOS TOKEN으로 채워진 디코더 입력 생성********************
        decoder_input = torch.full((batch_size,), SOS_TOKEN, dtype=torch.long, device=self.device)
        
        # 순회하면서 출력 단어를 생성합니다.
        # 0번째는 SOS TOKEN이 위치하므로, 1번째 인덱스부터 순회합니다.
        for t in range(0, output_length):
            # decoder_input : 디코더 입력 (batch_size) 형태의 SOS TOKEN로 채워진 입력
            # decoder_output: (batch_size, num_vocabs)
            # decoder_hidden: (Bidirectional(1) x number of layers(1), batch_size, hidden_size), context vector와 동일 shape
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)

            # t번째 단어에 디코더의 output 저장
            predicted_outputs[t] = decoder_output
            
            # teacher forcing 적용 여부 확률로 결정
            # teacher forcing 이란: 정답치를 다음 RNN Cell의 입력으로 넣어주는 경우. 수렴속도가 빠를 수 있으나, 불안정할 수 있음
            teacher_force = random.random() < teacher_forcing_ratio
            
            # top1 단어 토큰 예측
            top1 = decoder_output.argmax(1) 
            
            # teacher forcing 인 경우 ground truth 값을, 그렇지 않은 경우, 예측 값을 다음 input으로 지정
            decoder_input = outputs[:, t] if teacher_force else top1
        
        return predicted_outputs.permute(1, 0, 2) # (batch_size, sequence_length, num_vocabs)로 변경

In [141]:
#Seq2Seq 입출력 확인
# Encoder 정의
encoder = Encoder(num_vocabs=dataset.wordvocab.n_words, 
                       hidden_size=32, 
                       embedding_dim=64, 
                       num_layers=1)
# Decoder 정의
decoder = Decoder(num_vocabs=dataset.wordvocab.n_words, 
                       hidden_size=32, 
                       embedding_dim=64, 
                       num_layers=1)
# Seq2Seq 정의
seq2seq = Seq2Seq(encoder, decoder, 'cpu')

In [142]:
x, y = next(iter(train_loader))
print(x.shape, y.shape)
# (batch_size, sequence_length), (batch_size, sequence_length)

torch.Size([16, 25]) torch.Size([16, 25])


In [143]:
import random
output = seq2seq(x, y)
print(output.shape)
# (batch_size, sequence_length, num_vocabs)

torch.Size([16, 25, 886])


Training

In [144]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

NUM_VOCABS = dataset.wordvocab.n_words
HIDDEN_SIZE = 512
EMBEDDIMG_DIM = 256

print(f'num_vocabs: {NUM_VOCABS}\n======================')

# Encoder 정의
encoder = Encoder(num_vocabs=NUM_VOCABS, 
                  hidden_size=HIDDEN_SIZE, 
                  embedding_dim=EMBEDDIMG_DIM, 
                  num_layers=1)
# Decoder 정의
decoder = Decoder(num_vocabs=NUM_VOCABS, 
                  hidden_size=HIDDEN_SIZE, 
                  embedding_dim=EMBEDDIMG_DIM, 
                  num_layers=1)

# Seq2Seq 생성
# encoder, decoder를 device 모두 지정
model = Seq2Seq(encoder.to(device), decoder.to(device), device)
print(model)

num_vocabs: 886
Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(886, 256)
    (gru): GRU(256, 512)
  )
  (decoder): Decoder(
    (embedding): Embedding(886, 256)
    (dropout): Dropout(p=0.2, inplace=False)
    (gru): GRU(256, 512)
    (fc): Linear(in_features=512, out_features=886, bias=True)
  )
)


HyperParameter

In [146]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0, mode='min', verbose=True):
        """
        patience (int): loss or score가 개선된 후 기다리는 기간. default: 3
        delta  (float): 개선시 인정되는 최소 변화 수치. default: 0.0
        mode     (str): 개선시 최소/최대값 기준 선정('min' or 'max'). default: 'min'.
        verbose (bool): 메시지 출력. default: True
        """
        self.early_stop = False
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        
        self.best_score = np.Inf if mode == 'min' else 0
        self.mode = mode
        self.delta = delta
        

    def __call__(self, score):

        if self.best_score is None:
            self.best_score = score
            self.counter = 0
        elif self.mode == 'min':
            if score < (self.best_score - self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')
                
        elif self.mode == 'max':
            if score > (self.best_score + self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')
                
            
        if self.counter >= self.patience:
            if self.verbose:
                print(f'[EarlyStop Triggered] Best Score: {self.best_score:.5f}')
            # Early Stop
            self.early_stop = True
        else:
            # Continue
            self.early_stop = False

In [147]:
# 훈련에 적용할 하이퍼파라미터 설정

LR = 1e-3
optimizer = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()

es = EarlyStopping(patience=5, 
                   delta=0.001, 
                   mode='min', 
                   verbose=True
                  )

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min', 
                                                 factor=0.5, 
                                                 patience=2,
                                                 threshold_mode='abs',
                                                 min_lr=1e-8, 
                                                 verbose=True)

In [148]:
# train 함수 정의
def train(model, data_loader, optimizer, loss_fn, device):
    model.train()
    running_loss = 0
    
    for x, y in data_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        
        # output: (batch_size, sequence_length, num_vocabs)
        output = model(x, y)
        output_dim = output.size(2)
        
        # 1번 index 부터 슬라이싱한 이유는 0번 index가 SOS TOKEN 이기 때문
        # (batch_size*sequence_length, num_vocabs) 로 변경
        output = output.reshape(-1, output_dim)
        
        # (batch_size*sequence_length) 로 변경
        y = y.view(-1)
        
        # Loss 계산
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * x.size(0)
        
    return running_loss / len(data_loader)

In [149]:
# evaluation 함수 정의
def evaluate(model, data_loader, loss_fn, device):
    model.eval()
    
    eval_loss = 0
    
    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            output = model(x, y)
            output_dim = output.size(2)
            output = output.reshape(-1, output_dim)
            y = y.view(-1)
            
            # Loss 계산
            loss = loss_fn(output, y)
            
            eval_loss += loss.item() * x.size(0)
            
    return eval_loss / len(data_loader)

In [150]:
# 랜덤 샘플링 후 결과 추론
def sequence_to_sentence(sequences, index2word):
    outputs = []
    for p in sequences:

        word = index2word[p]
        if p not in [SOS_TOKEN, EOS_TOKEN, PAD_TOKEN]:
            outputs.append(word)
        if word == EOS_TOKEN:
            break
    return ' '.join(outputs)

In [151]:
# sequence를 다시 문장으로 바꾸어 문장 형식으로 출력하기 위한 함수

def random_evaluation(model, dataset, index2word, device, n=10):
    
    n_samples = len(dataset)
    indices = list(range(n_samples))
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # Sampling N indices
    
    # 샘플링한 데이터를 기반으로 DataLoader 생성
    sampler = SubsetRandomSampler(sampled_indices)
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)
    
    model.eval()
    with torch.no_grad():
        for x, y in sampled_dataloader:
            x, y = x.to(device), y.to(device)        
            output = model(x, y, teacher_forcing_ratio=0)
            # output: (number of samples, sequence_length, num_vocabs)
            
            preds = output.detach().cpu().numpy()
            x = x.detach().cpu().numpy()
            y = y.detach().cpu().numpy()
            
            for i in range(n):
                print(f'질문   : {sequence_to_sentence(x[i], index2word)}')
                print(f'답변   : {sequence_to_sentence(y[i], index2word)}')
                print(f'예측답변: {sequence_to_sentence(preds[i].argmax(1), index2word)}')
                print('==='*10)

In [152]:
#  훈련 시작
NUM_EPOCHS = 5
STATEDICT_PATH = 'seq2seq-chatbot-kor.pt'

best_loss = np.inf

for epoch in range(NUM_EPOCHS):
    loss = train(model, train_loader, optimizer, loss_fn, device)
    
    val_loss = evaluate(model, test_loader, loss_fn, device)
    
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), STATEDICT_PATH)
    
    if epoch % 5 == 0:
        print(f'epoch: {epoch+1}, loss: {loss:.4f}, val_loss: {val_loss:.4f}')
    
    # Early Stop
    es(loss)
    if es.early_stop:
        break
    
    # Scheduler
    scheduler.step(val_loss)
                   
model.load_state_dict(torch.load(STATEDICT_PATH))
torch.save(model.state_dict(), f'seq2seq-chatbot-kor-{best_loss:.4f}.pt')

epoch: 1, loss: 97.6070, val_loss: 48.2299
[EarlyStopping] (Update) Best Score: 97.60700
[EarlyStopping] (Update) Best Score: 73.25451
[EarlyStopping] (Update) Best Score: 69.53631
[EarlyStopping] (Update) Best Score: 67.74505
[EarlyStopping] (Update) Best Score: 66.44017


결과

In [153]:
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation(model, test_dataset, dataset.wordvocab.index2word, device)

질문   : 알츠하이머병 치료 를 위해 약물 이외 에 다른 치료법 들 이 존재 하 는지 알 고 싶 어요 .
답변   : 알츠하이머병 은 뇌 의 인지 기능 저하 가 주요 한 증상 을 보이 는 질병 입니다 . 이러 한 치매 는 시간 이 지날수록
예측답변: 알츠하이머병 은 치료 치료 는 치료 는 치료 는 의 치료 , , ,
질문   : 알츠하이머병 의 치료 에 는 어떤 종류 의 약물 이 사용 되 나요 ?
답변   : 알츠하이머병 은 현재 까지 근본 적 인 치료 방법 이 없 으며 , 현재 까지 알려진 유일 한 치료법 은 콜린 성 신경 전달
예측답변: 알츠하이머병 은 치료 치료 는 치료 는 치료 는 의 치료 , , ,
질문   : 알츠하이머병 치료 에 있 어서 어떤 치료 방법 이 가장 효과 적 일까요 ?
답변   : 알츠하이머병 은 기억 과 인지 기능 에 영향 을 주 는 만성 뇌 질환 으로 , 치료 와 관리 의 중요 성 이 있
예측답변: 알츠하이머병 은 치료 치료 는 치료 는 치료 는 의 치료 , , ,
질문   : 알츠하이머병 치료 에 있 어서 약물 이외 에 어떤 방법 이 효과 적 일까요 ?
답변   : 알츠하이머병 은 치매 의 가장 흔한 원인 이 며 , 아직 까지 근본 적 인 치료 방법 은 없 습니다 . 현재 까지 의
예측답변: 알츠하이머병 은 치료 치료 는 치료 는 치료 는 의 치료 , , ,
질문   : 알츠하이머병 치료 를 위해 어떤 약물 이 사용 될 수 있 는지 알려 주 세요 .
답변   : 알츠하이머병 은 가장 흔한 치매 로 , 뇌 의 퇴행 성 질환 입니다 . 초기 에 는 기억력 저하 로 시작 하 여 언어
예측답변: 알츠하이머병 은 치료 치료 는 치료 는 치료 는 의 치료 , , ,
질문   : 알츠하이머병 을 예방 하 기 위한 가족 들 의 주요 한 도움 방법 은 무엇 인가요 ?
답변   : 알츠하이머병 은 현재 까지 완전 한 치료 를 위한 치료법 은 없 지만 , 약물 치료 를 통해 일부 증상 을 완화 시킬 수
예측답변: 알츠하이머병 은 치료 치료 는 치료